## B11. Calculate basic statistics for DEM (std,avg,var,max,min, med) 

#### 0. Load required libraries, site dependant constants and utility functions.

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary (to have acces to the data and util catalogue)
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Create  necessary EOTasks

In [9]:
# define basic statistics calculation task:

class CalculateBasicStatisticsFromDataTimeless(EOTask): 
    
    """
    The task calculates avg, med, max, min, std, var from the timeless data.
    print(np.max(data))
    print(np.min(data))
    print(np.average(data))
    print(np.var(data))
    """
    
    def __init__(self, input_data_name, output_scalar_name, mode):
        self.input_data_name = input_data_name
        self.output_scalar_name = output_scalar_name
        self.mode = mode
        
    def execute(self, eopatch):
        
        data = eopatch.data_timeless[self.input_data_name].squeeze().reshape(333*333)

        if self.mode == 'min':
            scalar = np.min(data)
        elif self.mode == 'max':
            scalar = np.max(data)
        elif self.mode == 'avg':
            scalar = np.average(data)
        elif self.mode == 'med':
            scalar = np.median(data)
        elif self.mode == 'std':
            scalar = np.std(data)
        elif self.mode == 'var':
            scalar = np.var(data)
        else:
            print('No such a mode')
            
        eopatch.add_feature(FeatureType.META_INFO, self.output_scalar_name, scalar )
        
        return eopatch

#### 2. Create and execute EOWorkflow

In [11]:
dem_min_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_MIN', 'min') # name of output scalar
dem_max_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_MAX', 'max') # name of output scalar
dem_avg_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_AVG', 'avg') # name of output scalar
dem_med_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_MED', 'med') # name of output scalar
dem_std_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_STD', 'std') # name of output scalar
dem_var_eotask = CalculateBasicStatisticsFromDataTimeless('DEM_RAW_LAYER', 'DEM_RAW_VAR', 'var') # name of output scalar

# application of all eotasks for all eopatches in the a eoworkflow
for date_range_landsat in site.DATE_RANGES_LANDSAT_8 :
    print('Reading EOPatches from:', date_range_landsat)
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_landsat)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    dem_min_eotask,
    dem_max_eotask,
    dem_avg_eotask,
    dem_med_eotask,
    dem_std_eotask,
    dem_var_eotask,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()
    

Reading EOPatches from: ('2013-05-01', '2013-10-31')



Reading EOPatches from: ('2014-05-01', '2014-10-31')



Reading EOPatches from: ('2015-05-01', '2015-10-31')



Reading EOPatches from: ('2016-05-01', '2016-10-31')



Reading EOPatches from: ('2017-05-01', '2017-10-31')



Reading EOPatches from: ('2018-05-01', '2018-10-31')



Reading EOPatches from: ('2019-05-01', '2019-10-31')


## Check the variables

In [16]:
eo_patch = EOPatch.load('../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2019_05-2019_10/2969_maxcc_0.05_x-3_y-130')
eo_patch

EOPatch(
  data: {
    LANDSAT_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_GREEN_NIR: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_ICE: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_1: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_2: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    LANDSAT_QA_LAYERS: numpy.ndarray(shape=(23, 333, 333, 5), dtype=float32)
    LANDSAT_RAW_BANDS: numpy.ndarray(shape=(23, 333, 333, 11), dtype=float32)
    MODIS_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    MODIS_NDWI: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    MODIS_NRB: numpy.ndarray(shape=(23, 333, 333, 1), dtype=float32)
    MODIS_RAW_BANDS_DAY_0: numpy.ndarray(shape=(23, 333, 333, 7), dtype=float32)
    MODIS_RAW_BANDS_DAY_AFTER: numpy.nda